In [5]:
import numpy as np
import pandas as pd
from pandas.core.internals.construction import to_arrays
from prompt_toolkit.utils import to_int
from pulp import *

In [6]:
def find_loops(edges):
    """
    Find all closed loops starting and ending at node 0 in a directed graph.

    Args:
        edges: List of tuples representing directed edges (from_node, to_node)

    Returns:
        List of lists, where each sublist contains tuples forming a closed loop from 0 to 0
    """
    # Create adjacency list for faster lookup
    adj = {}
    for start, end in edges:
        if start not in adj:
            adj[start] = []
        adj[start].append(end)

    def find_paths(current, path, visited):
        """
        Recursive helper function to find all paths from current node back to 0.
        """
        if current == 'Depot' and len(path) > 0:  # Found a loop back to 0
            return [path[:]]

        if current not in adj:  # Dead end
            return []

        paths = []
        for next_node in adj[current]:
            if (current, next_node) not in visited:  # Avoid using same edge twice
                path.append((current, next_node))
                visited.add((current, next_node))
                paths.extend(find_paths(next_node, path, visited))
                path.pop()
                visited.remove((current, next_node))
        return paths

    # Start DFS from node 0
    all_loops = find_paths('Depot', [], set())

    # Filter out non-minimal loops (loops that contain other loops)
    minimal_loops = []
    for loop in all_loops:
        # Check if this loop contains any other loop
        is_minimal = True
        visited_nodes = {node for edge in loop for node in edge}
        for other_loop in all_loops:
            if loop != other_loop:
                other_nodes = {node for edge in other_loop for node in edge}
                if other_nodes.issubset(visited_nodes) and len(other_loop) < len(loop):
                    is_minimal = False
                    break
        if is_minimal:
            minimal_loops.append(loop)

    return minimal_loops

## Cost matrix

In [7]:
cost_matrix = pd.read_csv('../costs/costMatrixDistance.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDistance-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial-2020.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

## Shop demands

In [8]:
shop_demands = pd.read_csv('../2015_shop_locations.csv').set_index('id')[['demand(kg)', 'stage']].astype(float)
shop_demands = shop_demands.loc[shop_demands['stage'].isin([1, 2, 3, 4]), 'demand(kg)'] # Choose the stages to visit
shops = shop_demands.index
nodes = shops.copy()
nodes = np.append('Depot', nodes)

shop_demands.shape

(220,)

## Vehicle routing problem

In [9]:
# Vehicle capacity
van_capacity = 300

In [10]:
# Creates a list of tuples containing all the possible routes for transport
A = [(i,j) for i in nodes for j in nodes if i!=j]

In [11]:
A

[('Depot', 'ChIJwTC7A55ZwokRPNX9g7ngbaI'),
 ('Depot', 'ChIJU1XImaNZwokRutunetC8XeE'),
 ('Depot', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'),
 ('Depot', 'ChIJ5cPkuBtgwokRn55JgpGqjFA'),
 ('Depot', 'ChIJ24V7r_31wokR-S71l2zqrwc'),
 ('Depot', 'ChIJiUJ1DI5ZwokRWdK6SPg9BOY'),
 ('Depot', 'ChIJYTxm0Gf2wokR5V0iFDQ-2x0'),
 ('Depot', 'ChIJ_eQYpFdmwokR756MAH2tZPw'),
 ('Depot', 'ChIJYzaRC2REwokRaH2rHpflSYk'),
 ('Depot', 'ChIJ3R85DSb2wokR1XobETRMs5E'),
 ('Depot', 'ChIJ97BgiDtfwokRsbkP6Vl1seY'),
 ('Depot', 'ChIJT1ZEl4JYwokRXQMvGg9VMPE'),
 ('Depot', 'ChIJ7-uIlEFEwokRlbHGtjrdFtM'),
 ('Depot', 'ChIJU3IVO-tZwokRx6iWrhKQTng'),
 ('Depot', 'ChIJRR7W62BFwokRlrVks40mAdU'),
 ('Depot', 'ChIJnZ4lShdkwokRQUAn5mjU1pA'),
 ('Depot', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'),
 ('Depot', 'ChIJmZQsMgFbwokRMc5I7kPdQ7w'),
 ('Depot', 'ChIJt5DcjLRFwokRyxil5Pjp3Fo'),
 ('Depot', 'ChIJ2b9se6f1wokRUN8kWpFfzN0'),
 ('Depot', 'ChIJzRG6yrrzwokRQ5kjG2RJCwM'),
 ('Depot', 'ChIJ9X0DD_FhwokRIqyyecrKeMQ'),
 ('Depot', 'ChIJfxyTIQ32wokRxjovn0UE72M'),
 ('Depot', 

In [12]:
# A set of variables x is created to contain the vehicle routes variables (binary var)
x = {}
for a in A:
    x[a] = LpVariable("x(%s,%s)" %a, cat=LpBinary)

# A set of variables u is created to contain the continuous delivered quantity
u = {}
for shop in shops:
    u[shop] = LpVariable("u(%s)" %shop, shop_demands[shop], van_capacity)

In [13]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("CVRP_Problem",LpMinimize)

In [14]:
# The objective function is added to 'prob' first
prob += lpSum(cost_matrix.loc[i,j]*x[i,j] for (i,j) in A)

In [15]:
#Constraint 1 : imposes that exactly one arc enters each customer node
one_visit_out = {}
for shop in shops:
    one_visit_out[shop] = lpSum([x[shop,node] for node in nodes if shop!=node]) == 1.0
    prob += one_visit_out[shop]

In [16]:
#Constraint 2 : imposes that exactly one arc leaves each customer node
one_visit_in = {}
for shop in shops:
    one_visit_in[shop] = lpSum([x[node,shop] for node in nodes if node!=shop]) == 1.0
    prob += one_visit_in[shop]

In [17]:
#Constraint 4 : imposes the amount of flow that leaves the depot should be identical
#with the flow returns to the depot
depot_node = lpSum([x['Depot',shop] for shop in shops]) == lpSum([x[shop,'Depot'] for shop in shops])
prob += depot_node

In [18]:
#Constraint 5 : vehicle capacity constraint and sub tour elimination constraint
sub_tours_cap = {}
for shop_out in shops:
    for shop_in in shops:
        if shop_out != shop_in:
            sub_tours_cap[shop_out,shop_in] = u[shop_out] - u[shop_in] + van_capacity * x[shop_out,shop_in] <= van_capacity - shop_demands[shop_in]
            prob += sub_tours_cap[shop_out,shop_in]

In [19]:
# The problem is solved using PuLP's choice of Solver (ici Cplex, retourne 1 si résolu, -1 sinon)
prob.solve(GUROBI(msg=True, timeLimit=600))

Set parameter Username
Set parameter LicenseID to value 2612384
Academic license - for non-commercial use only - expires 2026-01-21
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12650H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 48621 rows, 48840 columns and 241780 nonzeros
Model fingerprint: 0x8bef6b99
Variable types: 220 continuous, 48620 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 3e+02]
  RHS range        [1e+00, 3e+02]
Presolve time: 0.37s
Presolved: 48621 rows, 48840 columns, 241780 nonzeros
Variable types: 220 continuous, 48620 integer (48620 binary)
Found heuristic solution: objective 3579.4858130
Found heuristic solution: objective 3577.619247

0

In [20]:
# The status of the solution is printed to the screen (optimal ou pas)
print("Status:", LpStatus[prob.status])

Status: Not Solved


In [21]:
# Each of the variables is printed with it's resolved optimum value
for key,val in x.items():
    if val.varValue==1.0:
        print(val,"=",val.varValue)

x(Depot,ChIJG7L_TLVbwokRT36uIrwz2Mo) = 1.0
x(Depot,ChIJG9cUYg_1wokRUNpvwM84Z4M) = 1.0
x(Depot,ChIJDchewklewokR8HKo3c8eLWY) = 1.0
x(Depot,ChIJXeLOsTZDwokRQ_l39cASWTQ) = 1.0
x(Depot,ChIJrY4KOzFawokRDCaABlBimbM) = 1.0
x(Depot,ChIJiWXDYR1cwokRL82F6r4W_bk) = 1.0
x(Depot,ChIJT6TbxYX2wokR17y5knj_EHI) = 1.0
x(Depot,ChIJefTurBlfwokRDQWW2f3cz84) = 1.0
x(Depot,ChIJS5sK5fxdwokRPLKKCZgMtHk) = 1.0
x(Depot,ChIJz_AjxCBewokRMpaMLTXBt4Q) = 1.0
x(Depot,ChIJUdrWy5bzwokRrqqaID530ss) = 1.0
x(Depot,ChIJccD1RIdZwokRnO2erj2tdag) = 1.0
x(Depot,ChIJ6RiUQ7eMwokRyhhHqH0yGG8) = 1.0
x(Depot,ChIJ34uYGnVbwokRv_kZPvMaKw0) = 1.0
x(Depot,ChIJsQODKt9ewokRUC_9IL1F9gw) = 1.0
x(Depot,ChIJEW6P5EJDwokRmLw7Ke_chn0) = 1.0
x(Depot,ChIJ7_c8yY5ZwokRKIArrMkYHmk) = 1.0
x(Depot,ChIJJYi2sUBZwokRfsq_XXpGvQw) = 1.0
x(Depot,ChIJQRY08qhbwokRke2RxC_TMmw) = 1.0
x(ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJiUJ1DI5ZwokRWdK6SPg9BOY) = 1.0
x(ChIJU1XImaNZwokRutunetC8XeE,ChIJf1Y5pQZZwokR_RLBsc_m67I) = 1.0
x(ChIJG7L_TLVbwokRT36uIrwz2Mo,ChIJwWywa4dCwokRAWd176K

In [22]:
# Print bounds
print("Lower bound = ", value(prob.solverModel.ObjBound))
print("Upper bound = ", value(prob.solverModel.ObjVal))

Lower bound =  370.6264068367713
Upper bound =  594.7551503722752


In [23]:
# Afficher les acrs correspondant à la solution
active_arcs = [key for key,val in x.items() if val.varValue >=0.9 ]
find_loops(active_arcs)


[[('Depot', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'),
  ('ChIJG7L-TLVbwokRT36uIrwz2Mo', 'ChIJwWywa4dCwokRAWd176KIs4U'),
  ('ChIJwWywa4dCwokRAWd176KIs4U', 'ChIJ6zAR6itowokRbSM46YIy-9o'),
  ('ChIJ6zAR6itowokRbSM46YIy-9o', 'ChIJN4cpNB1mwokRoz2EEGXJsPM'),
  ('ChIJN4cpNB1mwokRoz2EEGXJsPM', 'ChIJC085Xl9mwokRg0iipTP2kTA'),
  ('ChIJC085Xl9mwokRg0iipTP2kTA', 'ChIJywk2rk9hwokR7YWhekvNfgk'),
  ('ChIJywk2rk9hwokR7YWhekvNfgk', 'ChIJnZ4lShdkwokRQUAn5mjU1pA'),
  ('ChIJnZ4lShdkwokRQUAn5mjU1pA', 'ChIJ8TkuXuFjwokRBX2AiECzqHQ'),
  ('ChIJ8TkuXuFjwokRBX2AiECzqHQ', 'ChIJSUxHwVZYwokRXFyQi9V71Xw'),
  ('ChIJSUxHwVZYwokRXFyQi9V71Xw', 'ChIJi5-g-5VZwokRxp6Ytx_8D88'),
  ('ChIJi5-g-5VZwokRxp6Ytx_8D88', 'ChIJqVEEsyJawokRprbbV1yvNBE'),
  ('ChIJqVEEsyJawokRprbbV1yvNBE', 'Depot')],
 [('Depot', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'),
  ('ChIJG9cUYg_1wokRUNpvwM84Z4M', 'ChIJQRpR6Jj0wokRl9myJ19eEvU'),
  ('ChIJQRpR6Jj0wokRl9myJ19eEvU', 'ChIJB-hBRYr0wokRYXREBztnZ-o'),
  ('ChIJB-hBRYr0wokRYXREBztnZ-o', 'ChIJf9QMl6X0wokRESiH2zECSks'),
  ('ChIJf

In [24]:
# Each of the variables is printed with it's resolved optimum value
for key,val in u.items():
    print(val,"=",val.varValue)

u(ChIJwTC7A55ZwokRPNX9g7ngbaI) = 285.0
u(ChIJU1XImaNZwokRutunetC8XeE) = 138.0
u(ChIJG7L_TLVbwokRT36uIrwz2Mo) = 21.0
u(ChIJ5cPkuBtgwokRn55JgpGqjFA) = 135.0
u(ChIJ24V7r_31wokR_S71l2zqrwc) = 284.0
u(ChIJiUJ1DI5ZwokRWdK6SPg9BOY) = 300.0
u(ChIJYTxm0Gf2wokR5V0iFDQ_2x0) = 244.0
u(ChIJ_eQYpFdmwokR756MAH2tZPw) = 160.0
u(ChIJYzaRC2REwokRaH2rHpflSYk) = 220.0
u(ChIJ3R85DSb2wokR1XobETRMs5E) = 183.0
u(ChIJ97BgiDtfwokRsbkP6Vl1seY) = 300.0
u(ChIJT1ZEl4JYwokRXQMvGg9VMPE) = 281.0
u(ChIJ7_uIlEFEwokRlbHGtjrdFtM) = 271.0
u(ChIJU3IVO_tZwokRx6iWrhKQTng) = 242.0
u(ChIJRR7W62BFwokRlrVks40mAdU) = 204.0
u(ChIJnZ4lShdkwokRQUAn5mjU1pA) = 198.0
u(ChIJG9cUYg_1wokRUNpvwM84Z4M) = 27.0
u(ChIJmZQsMgFbwokRMc5I7kPdQ7w) = 114.0
u(ChIJt5DcjLRFwokRyxil5Pjp3Fo) = 129.0
u(ChIJ2b9se6f1wokRUN8kWpFfzN0) = 252.0
u(ChIJzRG6yrrzwokRQ5kjG2RJCwM) = 60.0
u(ChIJ9X0DD_FhwokRIqyyecrKeMQ) = 64.0
u(ChIJfxyTIQ32wokRxjovn0UE72M) = 267.0
u(ChIJoQK_Y1hnwokRLkhjEpVsGxo) = 245.0
u(ChIJ3eHXlHZZwokRuzME_28niVI) = 61.0
u(ChIJ0zAa911ZwokRQlmIU8v4Kzs)

In [27]:
# Convert the list of active arcs to a DataFrame
active_arcs_df = pd.DataFrame(active_arcs, columns=['From', 'To'])

# Save the DataFrame to a CSV file
active_arcs_df.to_csv('../routings/routing_optimal_2015_stage_4_Distance.csv', index=False)